## Especificação Tema Comum

O trabalho consiste em classificar cliente inadimplentes utilizando algoritmos de aprendizagem de máquina.
A base de dados contem 672 atributos e dados de clientes que contraíram empréstimos junto a instituição financeira. O que realizaram o pagamento em até 30 dias possuem valor zero na variável Y. Caso contrário, Y=1. A base contém 219984 registros.

Protocolo Experimental

A base deve ser dividida em 50%, 20% e 30% para treinamento, validação e teste, respectivamente. 
Note que a base é desbalanceada. Nesse caso, reporte métricas como Precision, Recall, F_Score e curva ROC.

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = '/content/gdrive/My Drive/Colab Notebooks/'

In [ ]:
os.chdir(path)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
 
from pylab import rcParams
 
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedBaggingClassifier
 
from collections import Counter

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/

## Data Import

In [ ]:
#abrir data
df = pd.read_csv('credit.csv')

In [ ]:
df

,ID,Y,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v33,v35,v37,v39,v41,v43,v45,...,v691,v692,v693,v694,v695,v696,v697,v698,v699,v700,v701,v702,v703,v704,v705,v706,v32,v34,v36,v38,v40,v42,v44,v46,v48,v50,v52,v54,v56,v58,v60,v62,v64,v66,v68,v70,v72,v74,v76,v78
0,151601,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,...,0.333333,1.0,549.0,1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,628.0,NaN,754.0,861.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
1,75091,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,...,1.000000,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,429.0,NaN,619.0,432.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
2,200877,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0.250000,3.0,7069.0,1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,378.0,NaN,640.0,620.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2255.00,450.00,300.0,1720.4
3,130751,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,...,0.000000,1.0,5447.0,1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,NaN,737.0,737.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
4,202571,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0.000000,3.0,582.0,1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,411.0,568.0,274.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,3076.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219979,205175,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,...,0.428571,8.0,7569.0,6,0.042948,0.0,0.042948,1.0,1.0,1.0,0.0,1.0,0.0,361.0,479.0,313.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
219980,42905,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.000000,3.0,4115.0,1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,397.0,287.0,329.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3045.21,1970.21,0.0,0.0
219981,87310,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.000000,4.0,2743.0,10,0.046337,0.0,0.046337,1.0,1.0,1.0,1.0,1.0,0.0,292.0,260.0,258.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,350.0,0.0,0.0,0.0,350.0,0.0,0.0,0.0,4141.00,366.00,0.0,650.0
219982,42906,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.000000,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,691.0,220.0,243.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0,600.0,0.0,0.0,600.0,600.0,0.0,0.0,600.00,600.00,0.0,0.0


## Transformação e exploração de dados

In [ ]:
for i in list(df):
    #print(dataset[i].median())
    median = df[i].median()
    df[i].fillna(median, inplace=True)
    #print(i, dataset[i].median())
    #break
    #print(i, dataset[i].mean())

In [ ]:
df = df.dropna(axis=1)

In [ ]:
# Qualquer são os valor ausente?
print(dataset.isnull().values.any())

# Número total de valores ausentes
print(dataset.isnull().sum().sum())

###Sobre o problema?
*Temos uma bas de dados com 672 atributos e dados de clientes.
Quem realizou o pagamento do empréstimo em 30 dias possuem um 0 na variável Y.
Preliminarmente os dados nos indica que temos 174.921 inadiplentes, mas quando pegamos o enunciado observamos uma detalhe muito importante. Os dados não estão balanciados isso nos custa uma falsa análise do que é real.

A melhor forma de trabalhar com dados desbalanceados na bibliografia é Usando regressão linear

In [ ]:
count_val=y_train_data['Y'].value_counts()

#Visualizando a contagem
X_ticks=['No default','Default']
bar_width=0.7
fig,ax=plt.subplots()
plt.bar(1,count_val[0],color='g', alpha=0.6)
plt.bar(2,count_val[1],color='r', alpha=0.6)
plt.text(1, count_val[0]/2, "{:,}".format(count_val[0]),color='darkgreen', ha='center', va= 'bottom',fontweight='bold',fontsize=12)
plt.text(2, count_val[1]/3, "{:,}".format(count_val[1]), color='darkred',ha='center', va= 'bottom',fontweight='bold',fontsize=12)
ax.set_title('Credito',fontsize=18,fontweight='bold')
ax.set_xlabel('Clientes',fontsize=16,fontweight='bold')
ax.set_ylabel('Pagamentos',fontsize=16,fontweight='bold')
ax.set_xticks([1,2])
ax.set_xticklabels(X_ticks,fontsize=14)
fig.tight_layout()
plt.show()

In [ ]:
#aplicando o teste
#Teste 0.2 = 20% ; 0.3 = 30 % e 0.5 = 50%
X_train, X_test, y_train, y_test =  train_test_split(dataset, y_train_data, test_size=0.2, random_state = 42)

In [ ]:
#otimizando dados desbalanceados
from imblearn.over_sampling import SMOTE

In [ ]:
print("Before OverSampling, counts of label '1': {}".format((y_train==1).sum()))
print("Before OverSampling, counts of label '0': {} \n".format((y_train==0).sum()))
sm = SMOTE(random_state=42)

In [ ]:
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)

In [ ]:
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format((y_train_res==1).sum()))
print("After OverSampling, counts of label '0': {}".format((y_train_res==0).sum()))

## Model treinamento e validação

*   Regressão logistica
*   SVM
*   Arvore de decisão 



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score, classification_report

def reduce_dimension(n, X):
    print("==============Dimensionality reduction using PCA==================")
    pca=PCA(n_components=n,whiten=True)
    pca.fit(X)
    training_r=pca.transform(X)
    print('{} of the variance has been retrieved for n={} in PCA'.format(pca.explained_variance_ratio_.sum(), n))
    return training_r


In [ ]:
X_train_res_r = reduce_dimension(20, X_train_res)
X_test_r = reduce_dimension(20, X_test)

###Regressão Logistica

In [ ]:
def fit_logistic_regression_with_linspace(X_train, y_train):
    parameters = { 'C': np.linspace(1, 10, 10) }
    lr = LogisticRegression()
    clf = GridSearchCV(lr, parameters, cv=5, verbose=5)
    clf.fit(X_train, y_train)
    return clf

# poor results with this technique
def do_logistic_regression_with_linspace(X_train_res, y_train_res, X_test, y_test):
    clf = fit_logistic_regression_with_linspace(X_train_res, y_train_res)
    print(clf.best_params_)
    # get best clf ?
    # lr1 = LogisticRegression(C=7, penalty='l2', verbose=5)
    # lr1.fit(X_train_res, y_train_res.values.ravel())
    # confusion_matrix(y_test, lr1.predict(X_test))
    # https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))

# with reduced dimension
#do_logistic_regression_with_linspace(X_train_res_r, y_train_res, reduce_dimension(20, X_test), y_test)

# 661 features, no reduce dimensions
do_logistic_regression_with_linspace(X_train_res, y_train_res, X_test, y_test)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] C=1.0 ...........................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............................... C=1.0, score=0.574, total=  29.5s
[CV] C=1.0 ...........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   29.5s remaining:    0.0s


[CV] ............................... C=1.0, score=0.576, total=  30.2s
[CV] C=1.0 ...........................................................


...

[CV] ............................... C=1.0, score=0.574, total=  29.7s
[CV] C=1.0 ...........................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  2.0min remaining:    0.0s


[CV] ............................... C=1.0, score=0.573, total=  30.6s
[CV] C=2.0 ...........................................................
[CV] ............................... C=2.0, score=0.576, total=  29.5s
[CV] C=2.0 ...........................................................
[CV] ............................... C=2.0, score=0.576, total=  29.9s
[CV] C=2.0 ...........................................................
[CV] ............................... C=2.0, score=0.575, total=  30.7s
[CV] C=2.0 ...........................................................
[CV] ............................... C=2.0, score=0.574, total=  29.5s
[CV] C=2.0 ...........................................................
[CV] ............................... C=2.0, score=0.574, total=  29.4s
[CV] C=3.0 ...........................................................
[CV] ............................... C=3.0, score=0.575, total=  30.1s
[CV] C=3.0 ...........................................................
[CV] .

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 24.8min finished


{'C': 8.0}
              precision    recall  f1-score   support

           0       0.27      0.46      0.34      8878
           1       0.84      0.69      0.76     35119

    accuracy                           0.65     43997
   macro avg       0.56      0.58      0.55     43997
weighted avg       0.72      0.65      0.67     43997



In [ ]:
model = do_logistic_regression_with_linspace(X_train, X_test, y_train, y_test)

In [ ]:
do_logistic_regression_with_linspace(X_train_res, y_train_res, X_test, y_test)

###SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

# not working - no convergence (too many data > 10000)
def fit_svm_using_gridsearch(training_r, y_train_res):
    # Create the parameter grid based on the results of random search 
    # params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
    #                     'C': [1, 10, 100, 1000]},
    #                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

    # Performing CV to tune parameters for best SVM fit 
    svm_model = GridSearchCV(LinearSVC(), params_grid, cv=5, verbose=5)
    # svm_model.fit(X_train_res, y_train_res.values.ravel())
    svm_model.fit(training_r, y_train_res)
    return svm_model

    svm_model(X_train_res, y_train_res, X_test, y_test) 

### Decision tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt  # doctest: +SKIP
from sklearn import datasets, metrics, model_selection, svm

def do_decision_tree(training_r, y_train, X_test, y_test):
    regr = DecisionTreeRegressor()
    regr.fit(training_r, y_train)
    print(confusion_matrix(y_test, regr.predict(X_test).round()))
    print(classification_report(y_test, regr.predict(X_test).round()))
    metrics.plot_roc_curve(y_test, regr.predict(X_test).round()))
    plt.show()  
    # why use predict().round() ? why regression is return number <> 0 and 1 ?
    for i in regr.predict(X_test):
        if (i != 0.0 and i != 1.0):
            print(i)
            
do_decision_tree(X_train_res, y_train_res, X_test, y_test)     

[[ 2907  5971]
 [ 7188 27931]]
              precision    recall  f1-score   support

           0       0.29      0.33      0.31      8878
           1       0.82      0.80      0.81     35119

    accuracy                           0.70     43997
   macro avg       0.56      0.56      0.56     43997
weighted avg       0.72      0.70      0.71     43997

0.8571428571428571
0.8571428571428571
0.3333333333333333
0.5
0.5
0.9166666666666666
0.9166666666666666
0.2564102564102564
0.9166666666666666
0.875
0.42857142857142855
0.875
0.42857142857142855
0.875
0.5
0.2564102564102564
0.42857142857142855
0.18181818181818182
0.2564102564102564
0.9411764705882353
0.9166666666666666
0.9411764705882353
0.8571428571428571
0.875
0.2564102564102564
0.875
0.9411764705882353
0.9411764705882353
0.6666666666666666
0.3333333333333333
0.9411764705882353


##Seleção de Caracteristicas
https://scikit-learn.org/stable/modules/feature_selection.html


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

def fit_using_SS_LinearSVC(training_r, y_train_res, X_test, y):
    SVCpipe = Pipeline([('scale', StandardScaler()),
                       ('SVC',LinearSVC())])
    # Gridsearch to determine the value of C
    param_grid = {'SVC__C':np.arange(0.01,100,10)}
    linearSVC = GridSearchCV(SVCpipe,param_grid,cv=5,return_train_score=True, verbose=5)
    linearSVC.fit(training_r,y_train_res)
    print(linearSVC.best_params_)    
    #linearSVC.coef_
    #linearSVC.intercept_
    print(classification_report(y_test, model.predict(X_test)))
    bestlinearSVC = linearSVC.best_estimator_
    bestlinearSVC.fit(X_train_res,y_train_res)
    bestlinearSVC.coef_ = bestlinearSVC.named_steps['SVC'].coef_
    print(bestlinearSVC.score(training_r,y_train_res))

fit_using_SS_LinearSVC(X_train_res, y_train_res, X_test, y_test) 

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] SVC__C=0.01 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... SVC__C=0.01, score=(train=0.638, test=0.630), total=10.1min
[CV] SVC__C=0.01 .....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 10.1min remaining:    0.0s


[CV] ..... SVC__C=0.01, score=(train=0.636, test=0.634), total=10.1min
[CV] SVC__C=0.01 .....................................................


...

[CV] ..... SVC__C=0.01, score=(train=0.635, test=0.638), total=10.0min
[CV] SVC__C=0.01 .....................................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 30.3min remaining:    0.0s


[CV] ..... SVC__C=0.01, score=(train=0.636, test=0.634), total=10.1min
[CV] SVC__C=0.01 .....................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 40.4min remaining:    0.0s


[CV] ..... SVC__C=0.01, score=(train=0.636, test=0.636), total=10.0min
[CV] SVC__C=10.01 ....................................................
[CV] .... SVC__C=10.01, score=(train=0.590, test=0.590), total= 9.9min
[CV] SVC__C=10.01 ....................................................
[CV] .... SVC__C=10.01, score=(train=0.547, test=0.546), total= 9.9min
[CV] SVC__C=10.01 ....................................................
[CV] .... SVC__C=10.01, score=(train=0.569, test=0.570), total= 9.7min
[CV] SVC__C=10.01 ....................................................
[CV] .... SVC__C=10.01, score=(train=0.523, test=0.526), total= 9.8min
[CV] SVC__C=10.01 ....................................................
[CV] .... SVC__C=10.01, score=(train=0.583, test=0.581), total= 9.8min
[CV] SVC__C=20.01 ....................................................
[CV] .... SVC__C=20.01, score=(train=0.587, test=0.579), total= 9.8min
[CV] SVC__C=20.01 ....................................................
[CV] .